**Traxxis 1/10 Scale RC Car Autonomous Vehicle Repositoy**

Demonstration code for running a small vehicle intersection with figure 8 loop using M1M1 LIDAR and SLAM for localization as well as an IMX160 camera mounted on 1/1/ scal Traxxis Slash remote control cars. Image recogition is done using Darkent YoloV4 Convolutional Neural Net trained to recognise our 1/10 scale CAVs. LIDAR point clustering is done with the DBScan Library. Local sensor fusion is don eusing a Kalman Filter and JPDA Filter. And global sensor fusion is done using a similar JPDA Filter and Kalman Filter combination.

**Installation:**
This first code snipped is simply downloading all the repositories from Github and installing all of the python library dependencies for them. We will be downloading and building the Darknet library so that we can use YoloV4 CNN to detect CAVs in video streams. We will then download the retrained YoloV4 Tiny models that are used for recognising our CAVs. Other python requirements will be downloaded and installed using PIP. And finally we will download a set of pre-recorded data from a test so that we can do some hardware in the loop testing using Colab in addition to our full simulation environment. Be aware this can take upwards of 5 minutes to execute. You should only need to run this a single time!

*Note for Physical Hardware Installation:*
To Install on physical Jetson hardware will not be as simple as running in Google Colab. We suggest first running this in Colab to get the hang of it before attempting a forray into installation on physical hardware. We will discuss these installation steps at the end of the Colab specific demo.

In [4]:
# Check our directory just in case
%cd /content/

# Clone 1/10th Scale Repo and Dependencies
!git clone https://github.com/eandert/One_Tenth_Scale_Autonomous_Vehicle.git
!cd One_Tenth_Scale_Autonomous_Vehicle && git checkout devel
!cd One_Tenth_Scale_Autonomous_Vehicle && pip install -r requirements.txt

# Clone Darknet and Dependencies
!git clone https://github.com/AlexeyAB/darknet/
!cd darknet/ && apt install libopencv-dev python-opencv ffmpeg

# Build darknet using CPU settings
!cd darknet && sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!cd darknet && sed -i 's/CUDNN=1/CUDNN=0/g' Makefile
!cd darknet && sed -i 's/GPU=1/GPU=0/g' Makefile
!cd darknet && sed -i 's/OPENMP=0/OPENMP=1/g' Makefile
!cd darknet && sed -i 's/AVX=0/AVX=1/g' Makefile
!cd darknet && sed -i 's/LIBSO=0/LIBSO=1/g' Makefile
!cd darknet && make
!chmod +x /content/darknet/libdarknet.so

# Download files from google drive for use with darknet
print(' Begining to download files... this may take a while...')

folders = ['/content/yolofiles', '/content/yolofiles/cav0', '/content/yolofiles/cav1']

files = [['/content/yolofiles/cav_test.txt', 'https://drive.google.com/u/0/uc?id=1WlL20udwRIoJq5Pkq1L6Iv6M3qhlc0MD&export=download'],
         ['/content/yolofiles/cav_train.txt', 'https://drive.google.com/u/0/uc?id=1bf0MobhUnzKnGjLdsBXGTs2vHOFIM2-w&export=download'],
         ['/content/yolofiles/cav.data', 'https://drive.google.com/u/0/uc?id=1jcEDFQ5n56Hq5tWJXXZ5N3Pg-p9qjq9t&export=download'],
         ['/content/yolofiles/cav.names', 'https://drive.google.com/u/0/uc?id=1hP7bfu5Ei-5w1cub5-vZJ-96mx0LLVMY&export=download'],
         ['/content/yolofiles/yolov4-tiny-cav.cfg', 'https://drive.google.com/u/0/uc?id=1yMQntYWsVbJ8h7x0r0Hv5y34o2IGRVJ2&export=download'],
         ['/content/yolofiles/yolov4-tiny-cav.weights', 'https://drive.google.com/u/0/uc?id=1g8r59Xcn5-n6jpc2ZjQdv-1jsLAiKN9w&export=download'],
         ['/content/yolofiles/cav0/cam_output.txt', 'https://drive.google.com/u/0/uc?id=1Vt5BEkVysTk5dJE0Lb47EQttT46LVB97&export=download'],
         ['/content/yolofiles/cav0/lidar_output.txt', 'https://drive.google.com/u/0/uc?id=1fTYHu3lmEIVjul30EfUyGWyvifPSSAQY&export=download'],
         ['/content/yolofiles/cav0/live_test_output.avi', 'https://drive.google.com/u/0/uc?id=1mqnQVoy8ZH2jDLZehC_J2ihtMcBfnse-&export=download'],
         ['/content/yolofiles/cav1/cam_output.txt', 'https://drive.google.com/u/0/uc?id=1xHz5YmlhRC0ex8PSE5IKB8ZA6DBaakDk&export=download'],
         ['/content/yolofiles/cav1/lidar_output.txt', 'https://drive.google.com/u/0/uc?id=1uYCHBluYurS4GGLSgPRpDsMWAY6GCaGb&export=download'],
         ['/content/yolofiles/cav1/live_test_output.avi', 'https://drive.google.com/u/0/uc?id=1kJnRCRyXbBuwu8RojMJgQy1PEngGmuXc&export=download']]


import os.path
import gdown

# Check that the folders exist or create them
for folder in folders:
  if not os.path.exists(folder):
    os.makedirs(folder)

# Check that the files exist or download them
for file in files:
  if os.path.isfile(file[0]): 
    print("File " +  file[0] + " found.")
  else:
    print("Downloading file " +  file[0] + ".")
    gdown.download(file[1], file[0], quiet=False)

# Fix a few lines in cav.data
!cd /content/yolofiles && sed -i 's/data\//\/content\/yolofiles\//g' cav.data

# We are done!
print("1/10 Scale CAV Simulation setup complete, proceed to next block.")

/content
fatal: destination path 'One_Tenth_Scale_Autonomous_Vehicle' already exists and is not an empty directory.
Branch 'devel' set up to track remote branch 'devel' from 'origin'.
Switched to a new branch 'devel'
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
fatal: destination path 'darknet' already exists and is not an empty directory.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
libopencv-dev is already the newest version (3.2.0+dfsg-4ubuntu0.1).
python-opencv is already the newest version (3.2.0+dfsg-4ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -ffp-contract=fast -mavx -

**Simulation:** To run the code in simulatuion mode (i.e. no real RC cars), simply run python main.py with the argument "simulation". Upon startup of the GUI you may enter target speeds for each of the 2 vehicles, this should be between 0 and 2 (units is meters per second) otherwise the control system will not be able to keep the vehicles on the track. Once there is a speed entered for both, click the "Start Test" button. The pause test will set the target speeds to 0 to temporarily pause the test. You can also update the target speeds in real time. Finally when you are done, click "End Test" and the simulation and all associated threads will be shut down correctly.

In [8]:
!pip install PyQt5
!cd One_Tenth_Scale_Autonomous_Vehicle && python main.py

     |████████████████████████████████| 8.3 MB 6.1 MB/s 
     |████████████████████████████████| 338 kB 45.5 MB/s 
     |████████████████████████████████| 59.9 MB 94 kB/s 
start 
Started FUSION successfully...
 * Serving Flask app "road_side_unit.src.communication" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Started LIDAR successfully...
 Vehicle  0  connecting to RSU...  127.0.0.1
 Vehicle  0  connected  127.0.0.1
RSU Initialized vehicle  0  thread
{'cav0': <Thread(Thread-2, started daemon 139893364709120)>}
Started LIDAR successfully...
 Vehicle  1  connecting to RSU...  127.0.0.1
RSU Initialized vehicle  1  thread
{'cav0': <Thread(Thread-2, started daemon 139893364709120)>, 'cav1': <Thread(Thread-3, started daemon 139893356316416)>}
 Vehicle  1  connected  127.0.0.1
Registered vehicle: 1 at 1643765228.9627821
127.0.0.1 - - [02/Feb/2022 01:27:08] "GET /RSU/register/ HT

Jetson Hardward Installation Guide (This is a work in progress!):
 Download the SD card image and upload to 64 GB SD card. This will save days of building the correct libraries, etc. TODO: Place google drive link here for Jetseon TX2 Clone this repo on the Home/Projects/ folder. TODO: Place google drive link here for Jetson Nano Clone this repo on the Home/Projects/ folder.

In order to recognise the 1/10th scale vehicles you will need the retrained version of YoloV4 tiny. Be sure to download and add the following YoloV4 files into the darknet folder:

In order to recognise the 1/10th scale vehicles you will need the retrained version of YoloV4 tiny. Be sure to download and add the following YoloV4 files into the darknet folder: 
 * [yolov4-tiny-cav.cfg](https://drive.google.com/file/d/1yMQntYWsVbJ8h7x0r0Hv5y34o2IGRVJ2/view?usp=sharing), needs to be located in /cfg
 * [cav.names](https://drive.google.com/file/d/1hP7bfu5Ei-5w1cub5-vZJ-96mx0LLVMY/view?usp=sharing), needs to be located in /data
 * [cav.data](https://drive.google.com/file/d/1jcEDFQ5n56Hq5tWJXXZ5N3Pg-p9qjq9t/view?usp=sharing), needs to be located in /data
 * [yolov4-tiny-cav.weights](https://drive.google.com/file/d/1g8r59Xcn5-n6jpc2ZjQdv-1jsLAiKN9w/view?usp=sharing), needs to be located in /weights

If you are interested in the data that was used to build this, you can find the 416x416 pictures converted for darknet here: [Training Data](https://drive.google.com/drive/folders/1pw01WHVJSjuO1fQmrj9-hd2wD756w0JB?usp=sharing). Over time this will increase in size as we add more data.